<a href="https://colab.research.google.com/github/salmanahmad10/BBAK/blob/main/Segmentation_Model_MainFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Preprocessing Module**

In [50]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

In [51]:
H = 128
W = 128

In [52]:
def process_data(data_path, file_path):
  df = pd.read_csv(file_path, header=None)
  names = df[0].values

  images = [os.path.join(data_path, f"Images/{name}") for name in names]
  masks = [os.path.join(data_path, f"Images/{name}") for name in names]

  return images, masks


In [53]:
def load_data(path):
  train_valid_path = os.path.join(path, "Images/Train.txt")
  test_path = os.path.join(path, "Images/Test.txt")

  train_x, train_y = process_data(path, train_valid_path)
  test_x, test_y = process_data(path, train_valid_path)

  train_x, valid_x = train_test_split(train_x, test_size = 0.1, random_state = 42)
  train_y, valid_y = train_test_split(train_y, test_size = 0.1, random_state = 42)

  return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [54]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [55]:
path = "gdrive/MyDrive/master/SharedGoogleDrive/Dataset/HandDrawnWireframes/Annotation/"

In [56]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

print(f"Dataset: Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

Dataset: Train: 202 - Valid: 23 - Test: 225


In [57]:
def readImage(x):
  x = cv2.imread(x, cv2.IMREAD_COLOR)
  x = cv2.resize(x, (W, H))
  x = x / 255.0
  x = x.astype(np.float32)

  return x

def readMask(x):
  x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(x, (W, H))
  x = x / 255.0
  x -= 1
  x = x.astype(np.int32)

  return x

In [58]:
def tf_dataset(x, y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x,y))
  dataset = dataset.shuffle(buffer_size=5000)
  dataset = dataset.map(preprocess)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  dataset = dataset.prefetch(2)
  return dataset

In [59]:

def preprocess(x, y):
  def f(x, y):
    x = x.decode()
    y = y.decode()

    image = readImage(x)
    mask = readMask(y)

    return image, mask

  image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
  mask = tf.one_hot(mask, 3, dtype=tf.int32)
  image = tf.ensure_shape(image, [H, W, 3])
  mask = tf.ensure_shape(mask, [H, W, 3])

  return image, mask

In [60]:
 (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)
 print(f"Dataset: Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")


Dataset: Train: 202 - Valid: 23 - Test: 225


In [61]:
dataset = tf_dataset(train_x, train_y, batch=8)

**Segmentation U-Net Model**

In [62]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model


In [63]:
def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x

In [64]:
def build_unet(shape, num_classes):
    inputs = Input(shape)

    """ Encoder """
    x1, p1 = conv_block(inputs, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)

    """ Bridge """
    b1 = conv_block(p4, 128, pool=False)

    """ Decoder """
    u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
    c1 = Concatenate()([u1, x4])
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
    c2 = Concatenate()([u2, x3])
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
    c3 = Concatenate()([u3, x2])
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
    c4 = Concatenate()([u4, x1])
    x8 = conv_block(c4, 16, pool=False)

    """ Output layer """
    output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

    return Model(inputs, output)

In [65]:
model = build_unet((128, 128, 3), 3)
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_247 (Conv2D)             (None, 128, 128, 16) 448         input_14[0][0]                   
__________________________________________________________________________________________________
batch_normalization_234 (BatchN (None, 128, 128, 16) 64          conv2d_247[0][0]                 
__________________________________________________________________________________________________
activation_234 (Activation)     (None, 128, 128, 16) 0           batch_normalization_234[0][0]    
___________________________________________________________________________________________

**Training**

In [66]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [67]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

In [68]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)
print(f"Dataset: Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

Dataset: Train: 202 - Valid: 23 - Test: 225


In [69]:
""" Hyperparameters """
shape = (128, 128, 3)
num_classes = 12
lr = 1e-4
batch_size = 8
epochs = 10

In [70]:
""" Model """
model = build_unet(shape, num_classes)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr))

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

callbacks = [
    ModelCheckpoint("model.h5", verbose=1, save_best_model=True),
    ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor="val_loss", patience=5, verbose=1)
]

model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_data=valid_dataset,
        validation_steps=valid_steps,
        epochs=epochs,
        callbacks=callbacks
)

Epoch 1/10


ValueError: ignored